In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:


import pandas as pd
import numpy as np
from privacypacking.utils.utils import LOGS_PATH
from experiments.ray.analysis import load_ray_experiment,load_latest_scheduling_results
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import copy



In [ ]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-03_10-09-16")
df = load_ray_experiment(logs)
df["alpha"] = df.config.apply(lambda x: x["omegaconf"]["alphas"][0])

In [ ]:
px.line(
    df.sort_values("alpha"),
    x="alpha",
    y="n_allocated_tasks",
    color="scheduler_metric",
    title="Number of allocated tasks for different monoalphas <br> Offline multiblock, mixed curves.",
    width=800,
)

In [ ]:
sdf = load_latest_scheduling_results(alphas="monoalpha")
# run_and_report_2022-02-03_10-01-17

In [ ]:
dominant_shares = sdf.groupby(["id", "alpha"])["normalized_epsilon"].agg(max)
dominant_shares = pd.DataFrame(dominant_shares).reset_index()
dominant_shares["dominant_shares"] = dominant_shares["normalized_epsilon"]
dominant_shares= dominant_shares.drop("normalized_epsilon", axis=1)


In [ ]:
dominant_shares.describe()

In [ ]:
merged = pd.merge(sdf, dominant_shares, on=["id", "alpha"])

In [ ]:
# Let's simplify a bit
merged = merged.query("alpha in [4,16,64]")

In [ ]:
px.scatter(
    # df,
    merged.sort_values(["id", "alpha"]).query("metric == 'Fcfs'"),
    x="id",
    y="normalized_epsilon",
    # range_y=[0,20],
#     color="log_id",
    color="metric",
    # barmode="group",
    # pattern_shape="allocated",
    log_y=True,
    range_x=[0,20],
    range_y=[1e-3, 1e5],
    facet_col="alpha",
    facet_col_wrap=1,
    # height=1200,
    width=500,
    title="Dominant shares of all tasks by order of arrival"
#     animation_frame="id"
)

In [ ]:
px.ecdf(
    # df,
    merged.sort_values(["id", "alpha"]).query("metric == 'Fcfs'"),
    x="normalized_epsilon",
    # y="normalized_epsilon",
    # range_y=[0,20],
#     color="log_id",
    color="metric",
    # barmode="group",
    # pattern_shape="allocated",
    log_x=True,
    # range_x=[0,20],
    # range_y=[1e-3, 1e5],
    facet_col="alpha",
    facet_col_wrap=1,
    # height=1200,
    width=500,
    title="CDF of Dominant shares of all tasks"
#     animation_frame="id"
)

In [ ]:
px.ecdf(
    # df,
    merged.sort_values(["id", "alpha"]).query("allocated"),
    x="normalized_epsilon",
    # y="normalized_epsilon",
    # range_y=[0,20],
#     color="log_id",
    color="metric",
    # barmode="group",
    # pattern_shape="allocated",
    log_x=True,
    # range_x=[0,20],
    # range_y=[1e-3, 1e5],
    facet_col="alpha",
    facet_col_wrap=1,
    # height=1200,
    width=800,
    title="CDF of Dominant shares of allocated tasks"
#     animation_frame="id"
)

In [ ]:
sdf.columns

In [ ]:
df.columns

In [ ]:
df.config.iloc[0]["omegaconf"]["alphas"][0]

In [ ]:
df["monoalpha"] = df.config.apply(lambda c: c["omegaconf"]["alphas"][0])

In [ ]:
df["tasks"].iloc[0]

In [ ]:
df["dominant_shares"] = df.tasks.apply()